In [28]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

# General imports
import numpy as np

from qiskit.quantum_info import SparsePauliOp

# custom module
from susy_qm import calculate_Hamiltonian

from scipy.optimize import minimize, differential_evolution
from scipy.stats.qmc import Halton

import json
import os

In [118]:
potential = 'DW'
cutoff = 8
shots = 1024

In [119]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [120]:
num_qubits

4

In [102]:
min_eigenvalue

np.float64(0.8845804438664059)

In [108]:
cnot_pool = []
cz_pool = []

for control in range(1,num_qubits):
        for target in range(1,num_qubits):
            if control != target:
                #cnot_pool.append(qml.CNOT(wires=[control, target]))
                cz_pool.append(qml.CZ(wires=[control, target]))

In [109]:
RY_pool = [qml.RY(0.0, wires=x) for x in range(num_qubits)]
RZ_pool = [qml.RZ(0.0, wires=x) for x in range(num_qubits)]
#RX_pool = [qml.RX(0.0, wires=x) for x in range(num_qubits)]
operator_pool =  RY_pool + cz_pool#cnot_pool

In [110]:
operator_pool

[RY(0.0, wires=[0]),
 RY(0.0, wires=[1]),
 RY(0.0, wires=[2]),
 RY(0.0, wires=[3]),
 CZ(wires=[1, 2]),
 CZ(wires=[1, 3]),
 CZ(wires=[2, 1]),
 CZ(wires=[2, 3]),
 CZ(wires=[3, 1]),
 CZ(wires=[3, 2])]

In [111]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def circuit(params, trial_op, op_list):

    param_index = 0
    if len(op_list) > 0:
        for o, _, w, _ in op_list:
            if (o == qml.CNOT) | (o == qml.CZ):
                o(wires=w)
            else:
                o(params[param_index], wires=w)
                param_index += 1

    op = type(trial_op)

    if (type(trial_op) == qml.CNOT) | (type(trial_op) == qml.CZ):
        op(wires=trial_op.wires)
    else:
        op(params[param_index], wires=trial_op.wires)
        param_index+=1
  
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [112]:
def cost_function(params, trial_op, op_list):
    
    params = pnp.tensor(params, requires_grad=True)
    energy = circuit(params, trial_op, op_list)

    return energy

In [114]:
num_steps = 10

#x0 = np.random.rand()

#variables
max_iter = 2000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-2
popsize = 15

op_list = []
min_params = []


for i in range(num_steps):

    print(f"step: {i}")

    energies = []
    e_params = []

    # Generate Halton sequence
    num_dimensions = len(op_list) + 1
    num_samples = popsize
    halton_sampler = Halton(d=num_dimensions)
    halton_samples = halton_sampler.random(n=num_samples)
    scaled_samples = 2 * np.pi * halton_samples

    bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
    x0 = min_params + [np.random.random()*2*np.pi]

    for trial_op in operator_pool:
        '''
        res = minimize(
                cost_function,
                x0,
                bounds=bounds,
                args=(trial_op, op_list),
                method= "COBYLA",
                options= {'maxiter':10000, 'tol': 1e-8}
            )
        
        '''
        res = differential_evolution(cost_function,
                                    bounds,
                                    args=(trial_op, op_list),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples
                                    )
        
        energies.append(res.fun)
        e_params.append(res.x)

    min_arg = np.argmin(energies)
    min_energy = energies[min_arg]
    print(f"Min energy: {min_energy}, {res.success}")

    min_op = type(operator_pool[min_arg])
    min_wires = operator_pool[min_arg].wires
    min_params = e_params[min_arg].tolist()

    if (i != 0):
        if np.abs(min_energy - op_list[-1][3]) < 1e-4:
            print("Converged")
            break

    if (i == 0):
        op_list.append((min_op, min_params, min_wires, min_energy))

    elif (i != 0) & (min_energy < op_list[-1][3]):
        op_list.append((min_op, min_params, min_wires, min_energy))
    elif (i != 0) & (min_energy >= op_list[-1][3]):
        print("Energy increased")
        #break


    

step: 0
Min energy: 0.9681129571874911, False
step: 1
Min energy: 0.9609480332140097, True
step: 2
Min energy: 0.9495945236546837, True
step: 3
Min energy: 0.9480930043891072, True
step: 4
Min energy: 0.9542087146189291, True
Energy increased
step: 5
Min energy: 0.9580870221248814, True
Energy increased
step: 6
Min energy: 0.9693351743048034, True
Energy increased
step: 7
Min energy: 0.9645673063570863, True
Energy increased
step: 8
Min energy: 0.9553377550135359, True
Energy increased
step: 9
Min energy: 0.9510674599863295, True
Energy increased


In [121]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue

    if o == qml.CNOT:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CNOT", w.tolist()))

    elif o == qml.CZ:
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append(("CZ",w.tolist()))

    elif o == qml.RY:
        last_operator[w[0]] = o
        reduced_op_list.append(("RY",w.tolist()))
        num_params+=1

    elif o == qml.RZ:
        last_operator[w[0]] = o
        reduced_op_list.append(("RZ",w.tolist()))
        num_params+=1

    elif o == qml.RX:
        last_operator[w[0]] = o
        reduced_op_list.append(("RX",w.tolist()))
        num_params+=1

Operator already added previously... skipping


In [122]:
reduced_op_list

[('RY', [3]), ('RY', [1]), ('CZ', [2, 3])]

In [13]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "COBYLA",
        "num steps": num_steps,
        "op_list": [str(x) for x in op_list],
        "reduced_op_list": reduced_op_list,
        "num_params": num_params}

In [42]:
data

{'potential': 'DW',
 'cutoff': 8,
 'optimizer': 'COBYLA',
 'num steps': 10,
 'op_list': ["(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, [5.433899017901948], Wires([3]), np.float64(1.1030338507414559))",
  "(<class 'pennylane.ops.op_math.controlled_ops.CZ'>, [5.152143850961136, 1.3771878196371807], Wires([0, 1]), np.float64(1.024796338274444))",
  "(<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RY'>, [5.335551943809088, -0.07314400864936067, 5.16995550773522], Wires([1]), np.float64(1.0147272651343762))"],
 'reduced_op_list': [('RY', [3]), ('CZ', [0, 1]), ('RY', [1])],
 'num_params': 2}

In [43]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for o, w in reduced_op_list:
        if o == "CNOT":
            qml.CNOT(wires=w)
        elif o == "CZ":
            qml.CZ(wires=w)
        elif o == "RY":
            qml.RY(params[params_index], wires=w)
            params_index +=1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [44]:
print(qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params))

0: ─╭●──────────────────┤ ╭<𝓗(M0)>
1: ─╰Z─────────RY(3.99)─┤ ├<𝓗(M0)>
2: ─────────────────────┤ ├<𝓗(M0)>
3: ──RY(5.22)───────────┤ ╰<𝓗(M0)>


In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\NoRotFiles\\"
os.makedirs(base_path, exist_ok=True)

cd = qml.draw(construct_ansatz, show_matrices=False)(reduced_op_list, params)

path = base_path + "cd_{}_{}.txt".format(potential, cutoff)
with open(path, 'w', encoding='utf-8') as f:
    f.write(cd)

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\NoRotFiles\\"
#base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
reduced_op_list

In [ ]:
qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=num_qubits)

In [ ]:
last_operator = {}
reduced_op_list = []
num_params = 0

for o, p, w, _ in op_list:

    if (o == qml.CNOT) | (o == qml.CZ):
        last_operator[w[0]] = o
        last_operator[w[1]] = o
        reduced_op_list.append((o,w))

    elif w[0] in last_operator.keys():
        if last_operator[w[0]] == o:
            print("Operator already added previously... skipping")
            continue
        else:
            last_operator[w[0]] = o
            reduced_op_list.append((o,w))
            num_params = num_params + 3
    else:
        last_operator[w[0]] = o
        reduced_op_list.append((o,w))
        num_params = num_params + 3

In [ ]:
params = np.random.uniform(0, 2 * np.pi, size=num_params)

dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def construct_ansatz(reduced_op_list, params):

    params_index = 0

    for i, (o, w) in enumerate(reduced_op_list):
        if (o == qml.CNOT) | (o == qml.CZ):
            o(wires=w)
        else:
            num_gate_params = o.num_params
            o(*params[params_index:(params_index + num_gate_params)], wires=w)
            params_index = params_index + num_gate_params

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))